!pip install "zarr<3" -U

In [1]:
from functoolz_bin import read_first_n_times, read_basic_to_xarray, list_of_time, units_from_base_time,set_time_encoding_hours 
from pathlib import Path
import numpy as np
import xarray as xr

## Initialization


### Convertion sbasic.dat into xarray

In [2]:
#Parameters 
path_to_sbasic="/mnt/c/Users/ecap/Documents/JCOPE/sbasic/sbasic.dat"
path_to_sbasic="/Users/todaka/data/jamstec/2103_ctl/sbasic.dat"
jcope_ctr_dir="/Users/todaka/data/jamstec/2103_ctl/"
jcope_bin_dir="/Users/todaka/data/jamstec/jcope2/"

out_path="/Users/todaka/data/jamstec/jcope2.zarr"
out_path="/Users/todaka/data/jamstec/jcope2.zarr"



# Example usage
times = list_of_time("202102220000", "202102282300")
#print(times)
#times=times[1:2]
times
initial_time="202102251600"
times = list_of_time(initial_time, "202102251700")

#initial_time="202102220000"
#path_to_sbasic="/scale/project/taos-s/public/jcope/jcope_bin/2103_ctl/sbasic.dat"
#jcope_ctr_dir="/scale/project/taos-s/public/jcope/jcope_bin/2103_ctl/"
#jcope_bin_dir="/scale/project/taos-s/public/jcope/jcope_bin/2102"
#out_path="/scale/project/taos-s/public/jcope/jcope_bin/zarr/tr.zarr"
#times = list_of_time(initial_time, "202102282300")

print(times)
times=times[1:2]
times

im: int = 902
jm: int = 650 
km: int = 47
print(im)

ds_sbasic, fildsc, ichflg = read_basic_to_xarray(path_to_sbasic,im=im,jm=jm,km=km,)

#ds_sbasic.chunk({"longitude":902/2,"latitude":650/2,"zlev":"10M"}).to_zarr('jcope.zarr',mode='w')
ds_sbasic

['202102251600', '202102251700']
902
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]


<xarray.Dataset> Size: 331MB
Dimensions:    (latitude: 650, longitude: 902, z: 47, zlev: 47)
Coordinates:
  * longitude  (longitude) float64 7kB 120.0 120.0 120.0 ... 145.0 145.0 145.0
  * latitude   (latitude) float64 5kB 16.99 17.01 17.04 ... 34.96 34.99 35.01
  * z          (z) int32 188B 1 2 3 4 5 6 7 8 9 ... 39 40 41 42 43 44 45 46 47
  * zlev       (zlev) float64 376B -2.5 -7.5 -12.5 ... -1.45e+03 -1.55e+03
Data variables:
    Z          (latitude, longitude, z) float32 110MB 0.0 -2.0 ... -5.803e+03
    ZZ         (latitude, longitude, z) float32 110MB -1.0 -3.5 ... -5.803e+03
    DZ         (latitude, longitude, z) float32 110MB 2.0 3.0 ... 269.0 269.0
Attributes: (12/13)
    title:                basic.dat (Z, ZZ, DZ) from a Fortran unformatted se...
    fortran_unformatted:  sequential
    endianness:           big-or-little (auto-detected)
    record_marker_bytes:  4
    fildsc:               runz
    im:                   902
    ...                   ...
    km:                   47
    dx:                   0.027777777777777776
    dy:                   0.027777777777777776
    xlons:                119.986111
    ylats:                16.986111
    ichflg_expected:      123456

Now we have converted the sbasic into xarray.dataset we have Z , ZZ (layers thickness), DZ 

zlev(kzd) →  layer centers

z_z(kzd+1) → depths of the interfaces.

dz_z(kzd) → vertical thickness of layers.

## Transform  first time step to zarr ( initialise the Zarr file)

In [6]:
tt_ctl = Path(jcope_ctr_dir) / "TT.ctl"
egt_ctl = Path(jcope_ctr_dir) / "EGT.ctl"
egtdir = Path(jcope_bin_dir) 
ttdir = Path(jcope_bin_dir) 
chunk={"time": 1, "latitude": "auto", "longitude": "5M","z":-1}



n_time=1
#ttdir
#ds_egt = read_first_n_times(egtdir, egt_ctl, "EGT_*.bin", n=n_time, no_z=True)
# -------------------

# Lire les n premiers pas de temps 

ds_egt = read_first_n_times(egtdir, egt_ctl, "EGT_"+initial_time+".bin", n=n_time, no_z=True)

ds_tt  = read_first_n_times(ttdir, tt_ctl,  "TT_"+initial_time+".bin", n=n_time)
ds_combined_ssh_tt = xr.merge([ds_egt, ds_tt])
ds_combined_ssh_tt = ds_combined_ssh_tt.rename({"x": "longitude", "y": "latitude", "sigma": "z"})
ds_combined_ssh_tt=ds_combined_ssh_tt.transpose("time", "latitude", "longitude", "z")

# Replace coords directly
ds_sbasic = ds_sbasic.assign_coords(
    longitude = ds_combined_ssh_tt.longitude,
    latitude  = ds_combined_ssh_tt.latitude
)

ds=xr.merge([ds_combined_ssh_tt, ds_sbasic]).chunk(chunk)
ds=ds.rename_vars({"Z":"_Z"})
# ds0 は最初の 1 ステップ（あなたの ds0）を想定
base_time = ds['time'].values[0]                   # ここが「イニシャルタイム」
units = units_from_base_time(base_time)            # 例: "hours since 2021-02-22 00:00:00"
ds = set_time_encoding_hours(ds, units)
time_encoding={'time': {'units': units}}

ds.to_zarr(out_path, mode='w',zarr_version=2 ,    consolidated=True,
    encoding=time_encoding
)
ds

True
/Users/todaka/data/jamstec/jcope2
[PosixPath('/Users/todaka/data/jamstec/jcope2/EGT_202102251600.bin')]
Reading EGT_202102251600.bin
False
/Users/todaka/data/jamstec/jcope2
[PosixPath('/Users/todaka/data/jamstec/jcope2/TT_202102251600.bin')]
Reading TT_202102251600.bin


/var/folders/1c/q1jqr0h541n720bvcqb_0rsm001mmz/T/ipykernel_36973/1827927097.py:37: FutureWarning: zarr_version is deprecated, use zarr_format
  ds.to_zarr(out_path, mode='w',zarr_version=2 ,    consolidated=True,


<xarray.Dataset> Size: 443MB
Dimensions:    (latitude: 650, longitude: 902, time: 1, z: 47, zlev: 47)
Coordinates:
  * latitude   (latitude) float64 5kB 16.99 17.01 17.04 ... 34.96 34.99 35.01
  * longitude  (longitude) float64 7kB 120.0 120.0 120.0 ... 145.0 145.0 145.0
  * time       (time) datetime64[ns] 8B 2021-02-25T16:00:00
  * z          (z) float64 376B 1.0 2.0 3.0 4.0 5.0 ... 43.0 44.0 45.0 46.0 47.0
  * zlev       (zlev) float64 376B -2.5 -7.5 -12.5 ... -1.45e+03 -1.55e+03
Data variables:
    el         (time, latitude, longitude) float32 2MB dask.array<chunksize=(1, 650, 902), meta=np.ndarray>
    t          (time, latitude, longitude, z) float32 110MB dask.array<chunksize=(1, 163, 163, 47), meta=np.ndarray>
    _Z         (latitude, longitude, z) float32 110MB dask.array<chunksize=(163, 163, 47), meta=np.ndarray>
    ZZ         (latitude, longitude, z) float32 110MB dask.array<chunksize=(163, 163, 47), meta=np.ndarray>
    DZ         (latitude, longitude, z) float32 110MB dask.array<chunksize=(163, 163, 47), meta=np.ndarray>
Attributes:
    title:                   el first 1 times concatenated from GrADS binaries
    history:                 time coordinate set from filename
    time_units_hours_since:  hours since 2021-02-25 16:00:00

## Make for loop of time , and load data of that time, append it to the final zarr file  

In [9]:

for time in times:
#time='202102251700'
    print('appeding time',time)
    ds_egt = read_first_n_times(egtdir, egt_ctl, "EGT_"+time+".bin", n=n_time, no_z=True)
    
    ds_tt  = read_first_n_times(ttdir, tt_ctl,  "TT_"+time+".bin", n=n_time)
    ds = xr.merge([ds_egt, ds_tt])
    ds = ds.rename({"x": "longitude", "y": "latitude", "sigma": "z"})
    ds=ds.transpose("time", "latitude", "longitude", "z")
    ds=ds.chunk(chunk)
    ds = set_time_encoding_hours(ds, units)

    ds.to_zarr(out_path,  mode='a', append_dim='time',zarr_version=2 
               ,  consolidated=True,
              )


appeding time 202102251700
True
/Users/todaka/data/jamstec/jcope2
[PosixPath('/Users/todaka/data/jamstec/jcope2/EGT_202102251700.bin')]
Reading EGT_202102251700.bin
False
/Users/todaka/data/jamstec/jcope2
[PosixPath('/Users/todaka/data/jamstec/jcope2/TT_202102251700.bin')]
Reading TT_202102251700.bin


/var/folders/1c/q1jqr0h541n720bvcqb_0rsm001mmz/T/ipykernel_36973/2114274847.py:22: FutureWarning: zarr_version is deprecated, use zarr_format
  ds.to_zarr(out_path,  mode='a', append_dim='time',zarr_version=2


In [10]:
xr.open_zarr(out_path).time

<xarray.DataArray 'time' (time: 2)> Size: 16B
array(['2021-02-25T16:00:00.000000000', '2021-02-25T17:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 16B 2021-02-25T16:00:00 2021-02-25T17:00:00